Import data

In [44]:
import pandas as pd
import matplotlib.pyplot as plt


man_city = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\man_city.csv")
arsenal = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\arsenal.csv")
brighton = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\brighton.csv")
bromwich = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\brom.csv")
burnley = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\burnley.csv")
chelsea = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\chelsea.csv")
crystal = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\crystal_palace.csv")
everton = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\everton.csv")
fulham = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\fulham.csv")
leeds = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\leeds.csv")
leicester = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\leicester.csv")
liverpool = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\liverpool.csv")
man_u = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\man_u.csv")
newcastle = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\newcastle.csv")
sheffield = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\sheffield.csv")
southampton = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\southampton.csv")
spurs = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\spurs.csv")
villa = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\villa.csv")
west_ham = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\west_ham.csv")
wolves = pd.read_csv(r"C:\Users\monik\Downloads\premier league project\wolves.csv")

complete_goals = pd.concat([man_city, arsenal, brighton, bromwich, burnley, chelsea, crystal, everton, fulham, leeds, leicester, liverpool, man_u, newcastle, sheffield, southampton, spurs, villa, west_ham, wolves])

In [84]:
teams = pd.read_csv(r"C:\Users\monik\Downloads\teams.csv")

list_f = []
list_j = teams["GF"]
for p in list_j:
    list_f.append(p/38)
    
teams["gpg"] = list_f
print(teams)

              Squad  MP   W   D   L  GF  GA  GD  Pts    xG   xGA   xGD  \
0   Manchester City  38  27   5   6  83  32  51   86  73.3  31.4  42.0   
1    Manchester Utd  38  21  11   6  73  44  29   74  60.2  42.2  18.0   
2         Liverpool  38  20   9   9  68  42  26   69  72.6  45.3  27.3   
3           Chelsea  38  19  10   9  58  36  22   67  64.0  32.8  31.2   
4    Leicester City  38  20   6  12  68  50  18   66  56.0  47.7   8.3   
5          West Ham  38  19   8  11  62  47  15   65  53.9  48.3   5.6   
6         Tottenham  38  18   8  12  68  45  23   62  54.5  49.5   5.0   
7           Arsenal  38  18   7  13  55  39  16   61  53.5  44.3   9.2   
8      Leeds United  38  18   5  15  62  54   8   59  57.5  62.9  -5.4   
9           Everton  38  17   8  13  47  48  -1   59  47.2  52.0  -4.8   
10      Aston Villa  38  16   7  15  55  46   9   55  52.9  52.9   0.1   
11    Newcastle Utd  38  12   9  17  46  62 -16   45  41.0  54.0 -13.0   
12           Wolves  38  12   9  17  3

Offensive WAR (basic not adjusted)

Methodology:
The WAR metric was popularized by sabremetrics in baseball but has been applied to other sports such as basketball. The basic formula is based on calculating the number of points added by a player and dividing it by the number of points per win. This will be applied in a basic offensive form in OWAR for the English Premier League. I will take the number of goals and assists and divide by "goals per win" which in this case will be calculated through regression analysis (below).

Notes on "goals per win":

In baseball's WAR, the "runs per win" component is effectively the average number of runs a team must score to add a single win to their total. To calculate this for soccer and specifically the premier league, I will use linear regression to build a model using the win totals for every team in the Premier League in 2020-2021 as the input and those respective teams' goals as the output for the training data. Based on this model I will calculate the required number of goals to achieve 1 win.

In [85]:
from sklearn.linear_model import LinearRegression


wins = teams[["W"]]

gpg = teams[["gpg"]]

gls = teams[["GF"]]

model = LinearRegression()
model.fit(wins, gls)


slope = model.coef_

intercept = model.intercept_

gpw = (slope * 1) + intercept

## Here we have calculated goals per win which will be used for all following WAR metrics

In [69]:
## Pythagenpat Method for calculating runs per win applied to goals per win:
import sympy as sym


gf = 0
for i in teams["GF"]:
    gf += i
    
ga = 0
for r in teams["GA"]:
    ga += r
    

x, y = sym.symbols('x y')


0.5


In [86]:
print(gpw)

[[14.03109006]]


OWAR (basic)

In [87]:


def simple_OWAR_2020(player, gpw_):
    list1 = complete_goals["Player"]
    search = 0
    for i in range(len(list1)):
        if(list1[i] == player):
            search = i
    
    goals = complete_goals["Gls"][search]
    assists = complete_goals["Ast"][search]
    ga = goals + assists
    ret = ga/gpw_
    return ret

def sOWAR(goals, assists, gpw_):
    ga = goals + assists
    ret = ga/gpw_
    return ret

In [88]:
## Harry Kane 2020-2021 season:

sOWAR(23, 14, gpw)

array([[2.63700111]])

Advanced OWAR uses specific changes to the original OWAR formula to adjust for luck similar to the principles behind BBall Index's LEBRON metric. For example, the goals component is changed to adjust for shooting "luck" using the expected goals metric found on Sports Reference. 
The full formula(s):

goals added = (expected goals - penalties) + penalties scored by player + ((penalties drawn - penalties taken) * league average penalty conversion rate) + (assisted shots * league scoring rate on shots)

aOWAR = goals_added/goals per win